<a href="https://colab.research.google.com/github/LordLean/Checkers-Game/blob/main/Checkers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [10]:
class GameBoard(object):


  def __init__(self):
    self.board = np.zeros(shape=(8,8)).astype("int").tolist()
    self.player_count = 0


  def setup_board(self):
    # Start player 2 at "top" of board.
    colour = 2
    # Counter to switch to using player 1 identifier.
    counter = 0 
    ln = len(self.board)
    for row_index in range(ln):
      if self.is_even(row_index) and row_index != 4:
        self.board[row_index] = [colour,0,colour,0,colour,0,colour,0]
      elif not self.is_even(row_index) and row_index != 3:
        self.board[row_index] = [0,colour,0,colour,0,colour,0,colour]
      counter += 1
      if counter == 4:
        colour = 1


  def get_valid_moves(self, piece, direction):
    # Get row, column (indicies) from tuple object piece.
    row, column = piece
    # Get player 1 or 2.
    player = self.board[row][column]
    # End function if non player piece.
    if player == 0:
      return
    # Potential next moves list.
    next_move_list = []
    # Next row - dependent on player.
    next_row = None
    if direction == "+":
      next_row = row + 1
      next_next_row = row + 2
    elif direction == "-":
      next_row = row - 1
      next_next_row = row - 2
    # List to hold columns on either side.
    left_right = [column-1,column+1]
    # Loop through left right options.
    for next_col in left_right:
      if next_col in range(8):
        next_space = (next_row,next_col)
        # Check state of potential next square.
        # Case: Empty square.
        if self.whats_in_the_box(next_space) == 0:
          next_move_list.append(next_space)
        # Case: Opponent-Occupied square.
        # If next space is opponent and next next space is clear.
        elif self.whats_in_the_box(next_space) != player: # This could be refined to check if == opponent. 
          # Assign next next column indicies.
          next_next_col = None
          if next_col == column - 1:
            next_next_col = column - 2
          else:
            next_next_col = column + 2
          next_next_space = (next_next_row, next_next_col)
          if self.whats_in_the_box(next_next_space) == 0:
            # self.remove_piece(next_row,next_col)
            next_move_list.append(next_next_space)
    return next_move_list  


  def move_piece(self, original_space, next_space):
    player = self.board[original_space[0]][original_space[1]]
    if player not in [1,2]:
      raise Exception("Trying to move piece from empty space.") 
    self.remove_piece(original_space)
    self.board[next_space[0]][next_space[1]] = player 
    
  def remove_piece(self,space):
    self.board[space[0]][space[1]] = 0

  def whats_in_the_box(self,space):
    return self.board[space[0]][space[1]]

  def print_board(self):
   for row in self.board:
      print(row,"\n")

  def is_even(self,num):
    return (num % 2) == 0 

In [14]:
gb = GameBoard()
gb.setup_board()

In [15]:
valid_moves = gb.get_valid_moves((2,0),"+")
gb.move_piece((2,0),valid_moves[0])
gb.print_board()

[2, 0, 2, 0, 2, 0, 2, 0] 

[0, 2, 0, 2, 0, 2, 0, 2] 

[0, 0, 2, 0, 2, 0, 2, 0] 

[0, 2, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 

[1, 0, 1, 0, 1, 0, 1, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 



In [5]:
gb.setup_board()
gb.print_board()
print("\n\n")
gb.board[3][1] = 1
gb.print_board()

piece = (2,2)
print(piece)
print(gb.get_valid_moves(piece,"+"))


[2, 0, 2, 0, 2, 0, 2, 0] 

[0, 2, 0, 2, 0, 2, 0, 2] 

[2, 0, 2, 0, 2, 0, 2, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 

[1, 0, 1, 0, 1, 0, 1, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 




[2, 0, 2, 0, 2, 0, 2, 0] 

[0, 2, 0, 2, 0, 2, 0, 2] 

[2, 0, 2, 0, 2, 0, 2, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 

[1, 0, 1, 0, 1, 0, 1, 0] 

[0, 1, 0, 1, 0, 1, 0, 1] 

(2, 2)
[(4, 0), (3, 3)]


In [ ]:
row = 5
for col_index in range(len(gb.board[2])):
  print("row:{}, col:{}".format(row,col_index))
  print(gb.get_valid_moves((row,col_index),"-"))
  print("\n\n")
